<a href="https://colab.research.google.com/github/em-chiu/intersection_project/blob/main/EC%20Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import urllib.request
from nltk.corpus import stopwords
import re
import numpy as np
from collections import Counter
from sklearn import preprocessing
import string
nltk.download('averaged_perceptron_tagger') # to solve pos feature extract issue
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB

import pandas as pd # data processing
import csv
# import warnings
# warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from bs4 import BeautifulSoup
import sys
from nltk.tokenize import WordPunctTokenizer
import unidecode
import seaborn as sns

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/emilychiu/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/emilychiu/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/emilychiu/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [3]:
# import os
# data_dir = '../data/'
# or_fname = 'FTR_new_labels.csv'
# fname = data_dir+or_fname


# load the data to df, try1 works
df = pd.read_csv('FTR_labels_copy.csv', header=None, encoding = "ISO-8859-1")\
        .drop(0, axis=1)\
        .rename(columns={1: 'tweet'})

In [4]:
# preview data
df.head()

,tweet,2
0,tweet,label
1,Elle ne devait pas dÃ©gager la nazi ??? https:...,1
2,je dois faire un oral en anglais sur Hitler et...,0
3,@ambrassmoi_ A Ã§a ð¤ de ressembler Ã la cr...,0
4,"""Par curiositÃ©, jâai voulu Ã©couter les dis...",0


In [47]:
def tokenize(corpus):
    # create a pattern to extract words
    pattern = re.compile(r'\b\w\w+\b')
    return(re.findall(pattern, corpus))

In [51]:
tokenize(stringtweet)

TypeError: expected string or bytes-like object

In [5]:
# Drop first row 
# by selecting all rows from first row onwards
df = df.iloc[1: , :]

In [6]:
# view dimensions of dataset
df.shape
# 2857 instances and 2 attributes in dataset

(2856, 2)

In [7]:
# Rename column names
col_names = ['tweet', 'label']
df.columns = col_names
df.columns

Index(['tweet', 'label'], dtype='object')

In [8]:
# preview after renaming
df.head()

,tweet,label
1,Elle ne devait pas dÃ©gager la nazi ??? https:...,1
2,je dois faire un oral en anglais sur Hitler et...,0
3,@ambrassmoi_ A Ã§a ð¤ de ressembler Ã la cr...,0
4,"""Par curiositÃ©, jâai voulu Ã©couter les dis...",0
5,@tical10 @ExBarcaFan74523 @ConflitsFrance Mdr ...,1


In [9]:
# summary of data
df.info()
# no missing data
# Categorical variables have data type object

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2856 entries, 1 to 2856
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   tweet   2856 non-null   object
 1   label   2856 non-null   object
dtypes: object(2)
memory usage: 44.8+ KB


In [21]:
# # check frequency distribution of values in label variable
df.label.value_counts()

0    1929
1     927
Name: label, dtype: int64

In [20]:
df.head()

,tweet,label
1,Elle ne devait pas dÃ©gager la nazi ??? https:...,1
2,je dois faire un oral en anglais sur Hitler et...,0
3,@ambrassmoi_ A Ã§a ð¤ de ressembler Ã la cr...,0
4,"""Par curiositÃ©, jâai voulu Ã©couter les dis...",0
5,@tical10 @ExBarcaFan74523 @ConflitsFrance Mdr ...,1


In [18]:
# some entries were saved as float. This function gest rid of the problem.
def to_string(text):
    if type(text)==float:
        text = str(text)
    return text

In [50]:
stringtweet = to_string(df['tweet'])

In [22]:
df.head()

,tweet,label
1,Elle ne devait pas dÃ©gager la nazi ??? https:...,1
2,je dois faire un oral en anglais sur Hitler et...,0
3,@ambrassmoi_ A Ã§a ð¤ de ressembler Ã la cr...,0
4,"""Par curiositÃ©, jâai voulu Ã©couter les dis...",0
5,@tical10 @ExBarcaFan74523 @ConflitsFrance Mdr ...,1


In [20]:
# this assumes all 0s are first then 1s
# # create a list of corresponding labels
# notracist_labels = [0] * 1929 # length of nonracist labels + header
# racist_labels = [1] * 927
# all_labels = notracist_labels + racist_labels

In [ ]:
# processing pipeline before data split
# processed data fed into X & y
# removing stopwords is preprocessing step (maniputate tweet column)

In [40]:
X = df['tweet'] # list of tweets
y = df['label'] # list of labels of tweets

In [41]:
print(X[0:2]) # first two tweets

1    Elle ne devait pas dÃ©gager la nazi ??? https:...
2    je dois faire un oral en anglais sur Hitler et...
Name: tweet, dtype: object


In [42]:
# split X and y into training and testing sets
# randomizes data in splitting
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [43]:
from sklearn.feature_extraction.text import TfidfVectorizer
# can't run LG or MNB (models) until vectorize (or bow)

# Initialize our vectorizer (featurizer, turning text to numbers)
#only looking at numbers produced, creating features
# takes each tweet turning into feature vector, each column corresponds to word
vectorizer = TfidfVectorizer() # empty model

# fit vectorizer w/ train data (training data)
# This makes your vocab matrix, learns words in vocab
vectorizer.fit(X_train)

# This transforms your documents into vectors (numbers).
X_train = vectorizer.transform(X_train)
X_test = vectorizer.transform(X_test)

In [ ]:
# label encoding not needed with tfidf
# one hot encoding used for text entries in column (not done w/ NLP)

In [46]:
print(type(X_train))

<class 'scipy.sparse.csr.csr_matrix'>


In [35]:
# Initalize our model.
model = MultinomialNB(alpha=.05)


# Fit our model with our training data.
model.fit(X_train, y_train)


# Make new predictions of our testing data. 
y_pred = model.predict(X_test)


# Make predicted probabilites of our testing data
y_pred_proba = model.predict_proba(X_test)

# Evaluate our model
accuracy =  model.score(X_test, y_test)

# Print our evaluation metrics
print("Model Accuracy: %f" % accuracy)

Model Accuracy: 0.748252


In [36]:
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred, target_names=model.classes_))

              precision    recall  f1-score   support

           0       0.77      0.91      0.83       390
           1       0.68      0.40      0.50       182

    accuracy                           0.75       572
   macro avg       0.72      0.66      0.67       572
weighted avg       0.74      0.75      0.73       572



In [ ]:
from nltk import ngrams

In [28]:
# compile a RE into regex object to look for occurrences of same pattern inside target strings w/o rewriting it
# ALF adds to data file
def add_lexical_features(dataframe):
    urls = re.compile(r"http")
    ats = re.compile(r"@[a-zA-Z.]*")
    hashtags = re.compile(r"#[a-zA-Z]*")
    letters = re.compile(r"[a-zA-Z]")
    caps = re.compile(r"[A-Z]")
    fancy = [";",'\"','(','<<']

    nbr_characters = [len(s) for s in df.tweet]
    df['nbr_characters'] = pd.DataFrame(nbr_characters, index=df.index)

    nbr_words = [len(s.split()) for s in df.tweet] # to update after cleaning
    df['nbr_words'] = pd.Series(nbr_words, index=df.index)

    nbr_ats = [len(ats.findall(text)) for text in df.tweet]
    df['nbr_ats'] = pd.Series(nbr_ats, index=df.index)

    nbr_hashtags = [len(hashtags.findall(text)) for text in df.tweet]
    df['nbr_hashtags'] = pd.Series(nbr_hashtags, index=df.index)

    nbr_urls = [len(urls.findall(text)) for text in df.tweet]
    df['nbr_urls'] = pd.Series(nbr_urls, index=df.index)

    nbr_letters = [len(letters.findall(text)) for text in df.tweet]
    df['nbr_letters'] = pd.Series(nbr_letters, index=df.index)

    nbr_caps = [len(caps.findall(text)) for text in df.tweet]
    df['nbr_caps'] = pd.Series(nbr_caps, index=df.index)

    nbr_fancy = [sum(1 for c in text if c in fancy) for text in df.tweet]
    df['nbr_fancy'] = pd.Series(nbr_fancy, index=df.index)

    return dataframe

In [29]:
# extract features
lexical_features = add_lexical_features(df)

In [30]:
# check size - 2856 tweets, 10 features
lexical_features.shape

(2856, 10)

In [31]:
# convert features and labels to numpy arrays
X = lexical_features
Y = np.array(all_labels)

In [32]:
# encode target label
# Label Encoding is converting labels/words into numeric form. 
# #Label encoding doesn't affect the dimensionality of the data set.

# INSTANTIATE
le = preprocessing.LabelEncoder()
# fit/transform, encoding df
# use df.apply() to apply le.fit_transform to all columns
X_2 = X.apply(le.fit_transform)
X_2.head()

,tweet,label,nbr_characters,nbr_words,nbr_ats,nbr_hashtags,nbr_urls,nbr_letters,nbr_caps,nbr_fancy
1,2196,1,57,8,0,0,1,43,6,0
2,2735,0,103,18,0,0,0,82,1,0
3,1588,0,53,10,1,0,0,37,1,0
4,577,0,113,17,0,0,0,85,4,1
5,2027,1,82,9,3,0,0,50,6,0


In [37]:
# split X and y into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_2, Y, test_size = 0.2, random_state = 0)

In [40]:
# n gram feature
from sklearn.feature_extraction.text import CountVectorizer
# counts the number of times a token shows up in the document and uses this value as its weight
# create a bag of words for only unigrams
cv = CountVectorizer(analyzer = 'word',ngram_range=(1,1), stop_words='english')
# # create a bag of words for with unigrams and bigrams
# cv = CountVectorizer(analyzer = 'word',ngram_range=(1,2), stop_words='english')

# convert training data to bag of words
X_train_cv = cv.fit_transform(X_train)
X_test_cv = cv.transform(X_test)

# train naive bayes classifier
clf = MultinomialNB()
clf.fit(X_train_cv, y_train)

# create predictions
y_pred = clf.predict(X_test_cv)

# find f-1 score
score = f1_score(y_test, y_pred, average='micro')
print('F-1 score : {}'.format(np.round(score,4)))

ValueError: Found input variables with inconsistent numbers of samples: [10, 2284]

In [ ]:
# train model with different n-gram ranges
for N in range(1,11):
    
    # convert training data to bag of words
    cv = CountVectorizer(analyzer = 'word',ngram_range=(1,N), stop_words='english')
    X_train_cv = cv.fit_transform(X_train)
    X_test_cv = cv.transform(X_test)
    
    # train model and generate predictions
    clf = MultinomialNB()
    clf.fit(X_train_cv, y_train)
    y_pred = clf.predict(X_test_cv)
    
    # compute f-1 score
    score = np.round(f1_score(y_test, y_pred, average='micro'),4)
    print('F-1 score of model with n-gram range of {}: {}'.format((1,N), score))

In [39]:
# check the shape of X_train and X_test

X_train.shape, X_test.shape
# ((2284, 10), (572, 10))

((2284, 10), (572, 10))

In [ ]:
y_train.shape, y_test.shape
# ((2284,), (572,))

In [ ]:
# check data types in X_train

X_train.dtypes

In [ ]:
# display categorical variables
categorical = [col for col in X_train.columns if X_train[col].dtypes == 'O']

categorical

In [ ]:
# display numerical variables
numerical = [col for col in X_train.columns if X_train[col].dtypes != 'O']

numerical

In [ ]:
# print percentage of missing values in the categorical variables in training set
X_train[categorical].isnull().mean()

In [ ]:
# check missing values in categorical variables in X_test
X_test[categorical].isnull().sum()

In [ ]:
X_train[categorical].head()

In [ ]:
# # One-Hot encoding the categorical parameters using get_dummies() 
# one_hot_encoded_data = pd.get_dummies(df, columns = ['tweet', 'label'])
# print(one_hot_encoded_data)

#One-hot encoding is the representation of categorical variables as binary vectors. 
#Using one-hot encoding increases the dimensionality of the data set. 


In [ ]:
# encode variables with one-hot encoding in preprocessing

#df['tweet'] = -------.fit_transform(df['tweet'])


# X_train = encoder.fit_transform(X_train).toarray()
# X_test = X_train = encoder.fit_transform(X_train).toarray()

In [35]:
X_train.head()

NameError: name 'X_train' is not defined

In [ ]:
X_train.shape

In [ ]:
X_test.head()

In [ ]:
X_test.shape

In [ ]:
# feature scaling

In [ ]:
# model training
# train a MNB classifier on the training set

# instantiate the model
MNB = MultinomialNB()


# fit the model
MNB.fit(X_train, y_train)

In [ ]:
# run classifier using 10-fold cross validation
# runs cross validation on dataset to test whether model can generalise over whole dataset
# function returns list of one score per split, and average of scores can be calculated to provide a single metric value for dataset
MNB_scores = cross_val_score(MultinomialNB(), X_2, Y, scoring='accuracy', cv=10)
print(MNB_scores.mean())

In [ ]:
# predicting results
y_pred = MNB.predict(X_test)
y_pred

In [ ]:
from sklearn.metrics import accuracy_score, f1_score
#checking accuracy score
print('Model accuracy score: {0:0.4f}'. format(accuracy_score(y_test, y_pred)))
# y_test are true class labels and y_pred are predicted class labels in test-set

In [ ]:
# compare the train-set and test-set accuracy to check for overfitting
y_pred_train = MNB.predict(X_train)
y_pred_train
print('Training-set accuracy score: {0:0.4f}'. format(accuracy_score(y_train, y_pred_train)))

In [ ]:
# print the scores on training and test set
print('Training set score: {:.4f}'.format(MNB.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(MNB.score(X_test, y_test)))

In [ ]:
# compare against null accuracy score

In [ ]:
# Print the Confusion Matrix and slice it into four pieces
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

print('Confusion matrix\n\n', cm)

print('\nTrue Positives(TP) = ', cm[0,0])

print('\nTrue Negatives(TN) = ', cm[1,1])

print('\nFalse Positives(FP) = ', cm[0,1])

print('\nFalse Negatives(FN) = ', cm[1,0])

In [ ]:
# visualize confusion matrix with seaborn heatmap

cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
                                 index=['Predict Positive:1', 'Predict Negative:0'])

sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')

In [ ]:
# Classification report to evaluate the classification model performance
from sklearn.metrics import classification_report

print(classification_report(y_test, y_pred))

In [ ]:
TP = cm[0,0]
TN = cm[1,1]
FP = cm[0,1]
FN = cm[1,0]

In [36]:
# print classification accuracy, same as test set score & model accuracy score

classification_accuracy = (TP + TN) / float(TP + TN + FP + FN)
print('Classification accuracy : {0:0.4f}'.format(classification_accuracy))

NameError: name 'TP' is not defined

In [ ]:
# print classification error
classification_error = (FP + FN) / float(TP + TN + FP + FN)

print('Classification error : {0:0.4f}'.format(classification_error))

In [ ]:
# f1-score is always lower than accuracy measures, they embed precision and recall into their computation
print('F1 Score: %.3f' % f1_score(y_test, y_pred))